# DataFrames, Data Slicing and Visualization Test Notebook

This notebook demonstrates:
1. Creating and storing data in pandas DataFrames
2. Slicing and filtering the data in various ways
3. Visualizing the data to extract insights

Let's begin by importing the necessary libraries.

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set plot styles
plt.style.use('seaborn-whitegrid')
sns.set_palette('viridis')
%matplotlib inline

## Part 1: Creating and Storing Data in DataFrames

Let's create some sample data to work with. We'll create a dataset that represents sales data across different regions, products, and time periods.

In [ ]:
# Create sample data
np.random.seed(42)  # For reproducibility

# Define parameters
regions = ['North', 'South', 'East', 'West']
products = ['Product A', 'Product B', 'Product C']
months = pd.date_range(start='2023-01-01', periods=12, freq='M')

# Create empty lists to store data
data = []

# Generate data
for region in regions:
    for product in products:
        for month in months:
            # Base sales varying by region and product
            base_sales = {
                'North': {'Product A': 100, 'Product B': 150, 'Product C': 80},
                'South': {'Product A': 120, 'Product B': 90, 'Product C': 110},
                'East': {'Product A': 80, 'Product B': 120, 'Product C': 140},
                'West': {'Product A': 150, 'Product B': 100, 'Product C': 90}
            }
            
            # Add seasonal variation
            month_factor = 1 + 0.2 * np.sin(month.month / 12 * 2 * np.pi)
            
            # Add random noise
            noise = np.random.normal(1, 0.15)
            
            # Calculate sales
            sales = int(base_sales[region][product] * month_factor * noise)
            
            # Calculate costs (approximately 60% of sales with some variation)
            cost = int(sales * (0.6 + np.random.normal(0, 0.05)))
            
            # Add row to data
            data.append({
                'date': month,
                'region': region,
                'product': product,
                'sales': sales,
                'cost': cost,
                'profit': sales - cost
            })

# Create DataFrame
df = pd.DataFrame(data)

# Display the first few rows
df.head()

In [ ]:
# Basic information about the DataFrame
print(f"DataFrame Shape: {df.shape}")
print("\nData Types:")
print(df.dtypes)
print("\nSummary Statistics:")
df.describe()

## Part 2: Slicing and Filtering Data

Now let's demonstrate various ways to slice and filter the data.

In [ ]:
# 1. Filter by a single column
# Get data for the North region only
north_data = df[df['region'] == 'North']
print(f"North region data shape: {north_data.shape}")
north_data.head()

In [ ]:
# 2. Filter by multiple conditions
# Get data for Product A in the South region
product_a_south = df[(df['product'] == 'Product A') & (df['region'] == 'South')]
print(f"Product A in South region data shape: {product_a_south.shape}")
product_a_south.head()

In [ ]:
# 3. Filter by date range
# Get data for Q1 2023 (Jan-Mar)
q1_data = df[(df['date'] >= '2023-01-01') & (df['date'] <= '2023-03-31')]
print(f"Q1 data shape: {q1_data.shape}")
q1_data.head()

In [ ]:
# 4. Select specific columns
# Get only region, product, and profit columns
profit_by_region_product = df[['region', 'product', 'profit']]
profit_by_region_product.head()

In [ ]:
# 5. Sort data
# Get top 10 most profitable entries
top_profit = df.sort_values('profit', ascending=False).head(10)
top_profit[['date', 'region', 'product', 'profit']]

In [ ]:
# 6. Aggregation by group
# Calculate total sales, cost, and profit by region
region_summary = df.groupby('region').agg({
    'sales': 'sum',
    'cost': 'sum',
    'profit': 'sum'
}).reset_index()

# Add profit margin column
region_summary['profit_margin'] = (region_summary['profit'] / region_summary['sales'] * 100).round(2)
region_summary

In [ ]:
# 7. Pivot table
# Create pivot table of average profit by region and product
pivot_table = pd.pivot_table(
    df,
    values='profit',
    index='region',
    columns='product',
    aggfunc='mean'
)

# Round values for better readability
pivot_table = pivot_table.round(2)
pivot_table

In [ ]:
# 8. Time-based aggregation
# Monthly total sales
df['month'] = df['date'].dt.strftime('%Y-%m')
monthly_sales = df.groupby('month').agg({
    'sales': 'sum',
    'profit': 'sum'
}).reset_index()
monthly_sales

In [ ]:
# 9. Custom calculations
# Calculate profit contribution percentage by product
product_profit = df.groupby('product')['profit'].sum().reset_index()
total_profit = product_profit['profit'].sum()
product_profit['contribution_percentage'] = (product_profit['profit'] / total_profit * 100).round(2)
product_profit.sort_values('contribution_percentage', ascending=False)

## Part 3: Visualizing Data

Now let's create various visualizations to extract insights from the data.

In [ ]:
# 1. Bar chart: Total sales by region
plt.figure(figsize=(10, 6))
sns.barplot(x='region', y='sales', data=region_summary, palette='viridis')
plt.title('Total Sales by Region', fontsize=14)
plt.xlabel('Region', fontsize=12)
plt.ylabel('Total Sales', fontsize=12)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)

# Add value labels on top of bars
for i, v in enumerate(region_summary['sales']):
    plt.text(i, v + 500, f'{v:,}', ha='center', fontsize=10)
    
plt.tight_layout()
plt.show()

In [ ]:
# 2. Line chart: Monthly sales trend
plt.figure(figsize=(12, 6))
plt.plot(monthly_sales['month'], monthly_sales['sales'], marker='o', linewidth=2, markersize=8)
plt.title('Monthly Sales Trend', fontsize=14)
plt.xlabel('Month', fontsize=12)
plt.ylabel('Total Sales', fontsize=12)
plt.xticks(rotation=45, fontsize=10)
plt.yticks(fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# 3. Pie chart: Profit contribution by product
plt.figure(figsize=(8, 8))
plt.pie(
    product_profit['profit'], 
    labels=product_profit['product'],
    autopct='%1.1f%%',
    startangle=90,
    shadow=True,
    explode=[0.05, 0, 0],  # Slightly pull out the first slice
    colors=sns.color_palette('viridis', len(product_profit))
)
plt.title('Profit Contribution by Product', fontsize=14)
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle
plt.tight_layout()
plt.show()

In [ ]:
# 4. Heatmap: Average profit by region and product
plt.figure(figsize=(10, 6))
sns.heatmap(
    pivot_table,
    annot=True,
    fmt='.1f',
    cmap='YlGnBu',
    linewidths=0.5,
    cbar_kws={'label': 'Average Profit'}
)
plt.title('Average Profit by Region and Product', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# 5. Multiple line chart: Sales trend by product
# First, aggregate the data by month and product
product_monthly = df.groupby(['month', 'product'])['sales'].sum().reset_index()
product_monthly_pivot = product_monthly.pivot(index='month', columns='product', values='sales')

# Create the plot
plt.figure(figsize=(12, 6))

for product in products:
    plt.plot(product_monthly_pivot.index, product_monthly_pivot[product], marker='o', linewidth=2, label=product)

plt.title('Monthly Sales Trend by Product', fontsize=14)
plt.xlabel('Month', fontsize=12)
plt.ylabel('Total Sales', fontsize=12)
plt.xticks(rotation=45, fontsize=10)
plt.yticks(fontsize=10)
plt.legend(title='Product', fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# 6. Stacked bar chart: Sales composition by region
region_product_sales = df.groupby(['region', 'product'])['sales'].sum().reset_index()

plt.figure(figsize=(12, 6))
sns.barplot(x='region', y='sales', hue='product', data=region_product_sales, palette='viridis')
plt.title('Sales Composition by Region and Product', fontsize=14)
plt.xlabel('Region', fontsize=12)
plt.ylabel('Total Sales', fontsize=12)
plt.legend(title='Product', fontsize=10)
plt.tight_layout()
plt.show()

In [ ]:
# 7. Box plot: Profit distribution by region
plt.figure(figsize=(12, 6))
sns.boxplot(x='region', y='profit', data=df, palette='viridis')
plt.title('Profit Distribution by Region', fontsize=14)
plt.xlabel('Region', fontsize=12)
plt.ylabel('Profit', fontsize=12)
plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

In [ ]:
# 8. Scatter plot: Sales vs. Profit
plt.figure(figsize=(10, 6))
sns.scatterplot(x='sales', y='profit', hue='product', size='cost', 
                sizes=(20, 200), alpha=0.7, palette='viridis', data=df)
plt.title('Sales vs. Profit by Product', fontsize=14)
plt.xlabel('Sales', fontsize=12)
plt.ylabel('Profit', fontsize=12)
plt.legend(title='Product', fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Part 4: Advanced Analysis and Insights

In [ ]:
# 1. Calculate profit margin by region and product
profit_analysis = df.groupby(['region', 'product']).agg({
    'sales': 'sum',
    'cost': 'sum',
    'profit': 'sum'
}).reset_index()

profit_analysis['profit_margin'] = (profit_analysis['profit'] / profit_analysis['sales'] * 100).round(2)
profit_analysis = profit_analysis.sort_values('profit_margin', ascending=False)

print("Top 5 most profitable region-product combinations:")
profit_analysis.head(5)

In [ ]:
# 2. Visualize profit margins
plt.figure(figsize=(12, 6))
chart = sns.barplot(x='region', y='profit_margin', hue='product', data=profit_analysis, palette='viridis')
plt.title('Profit Margin by Region and Product', fontsize=14)
plt.xlabel('Region', fontsize=12)
plt.ylabel('Profit Margin (%)', fontsize=12)
plt.legend(title='Product', fontsize=10)
plt.grid(True, alpha=0.3, axis='y')

# Add value labels
for p in chart.patches:
    chart.annotate(f'{p.get_height():.1f}%', 
                 (p.get_x() + p.get_width() / 2., p.get_height()), 
                 ha = 'center', va = 'bottom', 
                 xytext = (0, 5), textcoords = 'offset points')

plt.tight_layout()
plt.show()

In [ ]:
# 3. Time series analysis - Seasonal decomposition
# First, create a time series of total monthly sales
monthly_sales_ts = monthly_sales.set_index('month')['sales']

# Plot the time series
plt.figure(figsize=(12, 6))
monthly_sales_ts.plot()
plt.title('Monthly Sales Time Series', fontsize=14)
plt.xlabel('Month', fontsize=12)
plt.ylabel('Total Sales', fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Calculate month-over-month growth rate
monthly_sales['growth_rate'] = monthly_sales['sales'].pct_change() * 100
print("\nMonth-over-Month Growth Rate:")
monthly_sales[['month', 'sales', 'growth_rate']].dropna()

In [ ]:
# 4. Correlation analysis
# Look at correlation between sales, cost, and profit
correlation = df[['sales', 'cost', 'profit']].corr()

plt.figure(figsize=(8, 6))
sns.heatmap(correlation, annot=True, cmap='coolwarm', vmin=-1, vmax=1, linewidths=0.5)
plt.title('Correlation Matrix: Sales, Cost, and Profit', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# 5. Identify top performers and growth opportunities
# Identify the best and worst performing products by region
best_products = profit_analysis.loc[profit_analysis.groupby('region')['profit'].idxmax()]
worst_products = profit_analysis.loc[profit_analysis.groupby('region')['profit'].idxmin()]

print("Best performing products by region:")
best_products[['region', 'product', 'profit', 'profit_margin']]


In [ ]:
print("Worst performing products by region:")
worst_products[['region', 'product', 'profit', 'profit_margin']]

## Summary of Key Insights

Based on the analysis above, here are the key insights:

1. **Regional Performance**: The [insert best performing region] region has the highest total sales, while the [insert best profit margin region] region shows the best profit margins.

2. **Product Performance**: [Insert best product] contributes the most to overall profits, with a [insert percentage]% contribution.

3. **Seasonal Trends**: Sales show [describe seasonal pattern] throughout the year, with peaks in [insert peak months] and troughs in [insert trough months].

4. **Growth Opportunities**: The [insert region-product with low margin but high sales] combination shows potential for margin improvement, while [insert region-product with high margin but low sales] might benefit from increased marketing efforts to drive sales volume.

5. **Cost Structure**: There's a [strong/moderate/weak] correlation between sales and costs, indicating [efficient/inefficient] cost management across products and regions.

[Fill in the blanks based on the actual visualization results when running the notebook]